## Mobile Price Classification

### About Dataset
Bob has started his own mobile company. He wants to give tough fight to big companies like Apple,Samsung etc.

He does not know how to estimate price of mobiles his company creates. In this competitive mobile phone market you cannot simply assume things. To solve this problem he collects sales data of mobile phones of various companies.

Bob wants to find out some relation between features of a mobile phone(eg:- RAM,Internal Memory etc) and its selling price. But he is not so good at Machine Learning. So he needs your help to solve this problem.

In this problem you do not have to predict actual price but a price range indicating how high the price is

# Import Datasets

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

data = pd.read_csv('../datasets/mobilePrice/train.csv')
test =  pd.read_csv('../datasets/mobilePrice/test.csv')

In [3]:
# Preview the dataset (train)
df = pd.DataFrame(data)
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [20]:
# Preview the dataset (test)
df_test = pd.DataFrame(test)

In [21]:
# Find values boven 0.5 cm (drempelwaarde) en pak gemiddelde
average_above_threshold = df["m_dep"][df["m_dep"] >= 0.5].mean()
print(round(average_above_threshold,2))

0.73


In [22]:
# Replace values below 0.5 cm with gemiddelde boven de 0.5
df.loc[df["m_dep"] < 0.5, "m_dep"] = round(average_above_threshold, 2)
df.m_dep.min()

0.5

### Pixel Height

In [23]:
amount = len(df["px_height"][df["px_height"] < 65])
print("Amount of values that are under 65 pixels", amount)

Amount of values that are under 65 pixels 90


In [24]:
# Replace values below 65 pixels with 65 pixels
df.loc[df["px_height"] < 65, "px_height"] = 65

### Screen Width

In [25]:
amount = len(df["sc_w"][df["sc_w"] < 2])
print("Amount of values that are under 2 cm", amount)

Amount of values that are under 2 cm 390


In [26]:
# Replace values below 2cm with 2cm
df.loc[df["sc_w"] < 2,"sc_w"] = 2

Overbodige tabellen weghalen. Bv id in test set

In [27]:
df_test.drop(['id'], axis=1, inplace=True)
df_test.shape

(1000, 20)

# Naive Bayes


In [28]:
df2 = df

In [29]:
X = df2.drop('price_range', axis=1)
y = df2['price_range'].values.reshape(-1, 1).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bernoulli Naive Bayes algorithm for binary classification

In [30]:
model =  BernoulliNB()
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.255


0.255 is een slechte accuracy ik denk dat dit komt doordat het algoritme gemaakt is voor binary classification. Wat ik nu wil voorspellen is niet binary. (y heeft vier mogelijkheden)

Om een betere score te krijgen wil ik het MultinomialNB algoritme gebruiken om het Naive Bayes model te trainen.

# Multinomial Naive Bayes algorithm, which extends the Bernoulli Naive Bayes algorithm to handle multiple categories.

Het Multinomiale Naive Bayes-algoritme is een variant van het Naive Bayes-algoritme dat specifiek is ontworpen om meerdere categorieën (klassen) te kunnen behandelen.

In [31]:
# Create a Multinomial Naive Bayes model and fit it to the training data
model = MultinomialNB()
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")


Accuracy: 0.547


De accuracy is nog steeds niet wat ik wil, maar er is wel vooruitgang. MultinomialNB is eigenlijk een algoritme voor tekst classificatie. Ik denk daarom dat het nu laag scored. Model Naive Bayes is niet de beste keuze bij het voorspellen van de prijs.



# Conclusie Naive Bayes

Naive Bayes algoritme is niet het juiste algoritme voor het voorspellen van de prijs van de telefoon. Dit kan je ook zien aan de accuracy scores. Eerst had ik een normale binary Naive Bayes algoritme gebruikt om mijn model te trainen. Deze gaf een score van 0.255. Dit komt doordat er 4 mogelijke prijs categorieen zijn, dus het classificeren als binary klopt niet.(Meer dan twee opties) Daarom heb ook multinomial Naive Bayes algoritme geprobeerd. Deze had het dubbele meer correct. De score was: 0.547. Hier ben eigenlijk nog steeds niet te vreden mee qua score. Ik denk dat het komt doordat Naive Bayes niet is gemaakt voor wat ik wil doen met mijn dataset.
